In [1]:
# Titanic Competition: https://www.kaggle.com/competitions/titanic/data
import pandas as pd
import seaborn as sns

train = pd.read_csv('Dados/titanic/train.csv')
test = pd.read_csv('Dados/titanic/test.csv')

In [2]:
train = train.set_index('PassengerId')
test = test.set_index('PassengerId')

train.shape, test.shape


((891, 11), (418, 10))

In [3]:
# Add new feature call 'travel_with_family', 'size_family' and title, based on last name
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

name_split_train = train['Name'].str.split(',').str
title = name_split_train[1].str.split('.').str[0]
train['Surname'] = name_split_train[0]
train['title'] = title
train['size_family'] = train['Surname'].map(train['Surname'].value_counts()).astype(int)
train['travel_with_family'] = train['size_family'] > 1

name_split_test = test['Name'].str.split(',').str
title_test = name_split_test[1].str.split('.').str[0]
test['Surname'] = name_split_test[0]
test['title'] = title_test
test['size_family'] = test['Surname'].map(test['Surname'].value_counts())
test['travel_with_family'] = test['size_family'] > 1

map_travel_with_family = {False: 0, True: 1}
test['travel_with_family'] = test['travel_with_family'].map(map_travel_with_family)
train['travel_with_family'] = train['travel_with_family'].map(map_travel_with_family)


# Encoding Title
# enc = OrdinalEncoder()
# cabin_enc = enc.fit_transform(train[['title']])
# train['title'] = cabin_enc[:, 0]

# cabin_enc_test = enc.transform(test[['title']])
# test['title'] = cabin_enc_test[:, 0]

rare_titles = ['Dona', 'Lady', 'the Countess','Capt', 'Col', 'Don', 
                   'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Done']
train['title'] = train['title'].str.strip()
test['title'] = test['title'].str.strip()
train['title'] = train['title'].replace(rare_titles, 'Rare')
test['title'] = test['title'].replace(rare_titles, 'Rare')

train['title'] = train['title'].replace('Mlle', 'Miss')
train['title'] = train['title'].replace('Ms', 'Miss')
train['title'] = train['title'].replace('Mme', 'Mrs')
test['title'] = test['title'].replace('Mlle', 'Miss')
test['title'] = test['title'].replace('Ms', 'Miss')
test['title'] = test['title'].replace('Mme', 'Mrs')

ohe = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

titles_ohe_train = ohe.fit_transform(train[['title']])
titles_ohe_test = ohe.transform(test[['title']])

train = pd.concat([train.drop(columns=['title']), titles_ohe_train], axis=1)
test = pd.concat([test.drop(columns=['title']), titles_ohe_test], axis=1)

train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,size_family,travel_with_family,title_Master,title_Miss,title_Mr,title_Mrs,title_Rare
PassengerId,,,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,2,1,0.0,0.0,1.0,0.0,0.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,1,0,0.0,0.0,0.0,1.0,0.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,1,0,0.0,1.0,0.0,0.0,0.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Futrelle,2,1,0.0,0.0,0.0,1.0,0.0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Allen,2,1,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Montvila,1,0,0.0,0.0,0.0,0.0,1.0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Graham,3,1,0.0,1.0,0.0,0.0,0.0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Johnston,2,1,0.0,1.0,0.0,0.0,0.0


In [4]:
# To complete the cabin, lets suppose that the family was in same cabin.
    
nan_cabin_passenger = train.loc[train['Cabin'].isnull() 
                                & train['travel_with_family'] == 1][
                                ['Surname', 'travel_with_family', 'Name', 'Cabin']
                                ]
cabin_family = train.loc[~train['Cabin'].isnull() & train['Surname'].isin(nan_cabin_passenger['Surname'])][['Name', 'Surname', 'Cabin']]
mapping_surname_cabin = cabin_family.drop_duplicates('Surname')
mapping_surname_cabin = mapping_surname_cabin.set_index('Surname')['Cabin']
train['Cabin'] = train['Cabin'].fillna(train['Surname'].map(mapping_surname_cabin))

In [30]:
# Dealing with miss values about cabin and encoding the cabin
import numpy as np
import math
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder


def get_num_cabin(x):
    if not x:
        return 0
    val = []
    try:
        for v in x:
            val.append(int(v))
    except:
        return 0
    return sum(val) // len(val)
 
cabins_letter = train['Cabin'].astype(str).str.replace(r'[^a-zA-Z]', '', regex=True).str.split()
def choose_random(val):
    if 'nan' in val or '' in val:
        return 'U'
    
    if len(val) > 1:
        choice = np.random.choice(val)
        if len(choice) > 1:
            return choice[0]
            
        return choice
    if len(val[0]) > 1:
        return val[0][0]
        
    return val[0]

cabins_number = train['Cabin'].str.findall(r'\d+')
train['Cabin_Number'] = cabins_number.apply(get_num_cabin)
cabins_number = test['Cabin'].str.findall(r'\d+')
test['Cabin_Number'] = cabins_number.apply(get_num_cabin)

cabins_letter = cabins_letter.apply(choose_random)
train['Cabin'] = cabins_letter
cabins_letter_test = test['Cabin'].astype(str).str.replace(r'[^a-zA-Z]', '', regex=True).str.split()
cabins_letter_test = cabins_letter_test.apply(choose_random)
test['Cabin'] = cabins_letter_test

# enc = OrdinalEncoder()
# cabin_enc = enc.fit_transform(train[['Cabin']])
# train['Cabin'] = cabin_enc[:, 0]

# cabin_enc_test = enc.transform(test[['Cabin']])
# test['Cabin'] = cabin_enc_test[:, 0]

# le = LabelEncoder()
# train['Cabin'] = le.fit_transform(train['Cabin'])
# test['Cabin'] = le.transform(test['Cabin'])

ohe_cab = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

cabin_ohe_train = ohe_cab.fit_transform(train[['Cabin']])
cabin_ohe_test = ohe_cab.transform(test[['Cabin']])

train = pd.concat([train.drop(columns=['Cabin']), cabin_ohe_train], axis=1)
test = pd.concat([test.drop(columns=['Cabin']), cabin_ohe_test], axis=1)

KeyError: 'Cabin'

In [6]:
# Dealing with the missing values about embarked.
# I will put the mathematical mode.

mode_train = train['Embarked'].value_counts()
train['Embarked'] = train['Embarked'].fillna('S')

# Encoding "Embarked" 
map_embarked = {'S': 0, 'C': 1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(map_embarked)
test['Embarked'] = test['Embarked'].map(map_embarked)

In [7]:
train['Sex'] = train['Sex'].replace(to_replace=['male', 'female'], value=[0, 1])
test['Sex'] = test['Sex'].replace(to_replace=['male', 'female'], value=[0, 1])

/tmp/ipykernel_134922/4184202167.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Sex'] = train['Sex'].replace(to_replace=['male', 'female'], value=[0, 1])
/tmp/ipykernel_134922/4184202167.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Sex'] = test['Sex'].replace(to_replace=['male', 'female'], value=[0, 1])


In [8]:
#The 'ticket' column contains string data. The authors suggest using an encoding method for this data type. I am applying the Feature Hashing method,
#as described in Feature Engineering and Selection by Max Kuhn and Kjell Johnson

from sklearn.feature_extraction import FeatureHasher

features = 2**2
# features = 3

h = FeatureHasher(n_features=features, input_type='string')
ticket_proc = train['Ticket'].astype(str).str.replace('/', ' ', regex=False).str.replace(r'[^\w\s]', '', regex=True).str.split()
ticket_feat_hasher = h.fit_transform(ticket_proc).toarray()
ticket_feat_hasher_df = pd.DataFrame(ticket_feat_hasher, columns = [f'f{i}' for i in range(len(ticket_feat_hasher[0]))], index = train.index)
train = pd.concat([train, ticket_feat_hasher_df], axis=1)
train = train.drop(columns = ['Ticket'])


ticket_proc = test['Ticket'].astype(str).str.replace('/', ' ', regex=False).str.replace(r'[^\w\s]', '', regex=True).str.split()
ticket_feat_hasher = h.transform(ticket_proc).toarray()
ticket_feat_hasher_df = pd.DataFrame(ticket_feat_hasher, columns = [f'f{i}' for i in range(len(ticket_feat_hasher[0]))], index = test.index)
test = pd.concat([test, ticket_feat_hasher_df], axis=1)
test = test.drop(columns = ['Ticket'])

In [9]:
train['Age'].isna().sum() / train['Age'].isna().count() # 20% unknow, 80% know. Remove the unknow data is not appropriate

# Solution: Impute data with KNN. Source: Feature Engineering and Selection by Max Kuhn and Kjell Johnson
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

df_aux = train.select_dtypes(include = ['number'])

corr = df_aux.corr()['Age']
corr_abs = 0.15
corr_age_feat = corr[abs(corr) > corr_abs].index
# corr[abs(corr) > corr_abs], corr_age_feat

imputer = KNNImputer(n_neighbors=3)
data_to_impute = train[corr_age_feat]

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_to_impute)

imputer_df_pred = imputer.fit_transform(data_scaled) # KNN
imputed_data_final = scaler.inverse_transform(imputer_df_pred) # Return to normal scale

df_aux_train = pd.DataFrame(imputed_data_final, columns = corr_age_feat, index=train.index)

train['Age'] = df_aux_train['Age']


data_to_impute_test = test[corr_age_feat]
data_scaled_test = scaler.transform(data_to_impute_test)
imputer_df_pred_test = imputer.transform(data_scaled_test) # KNN
imputed_data_final_test = scaler.inverse_transform(imputer_df_pred_test) # Return to normal scale
df_aux_test = pd.DataFrame(imputed_data_final_test, columns = corr_age_feat, index=test.index)
test['Age'] = df_aux_test['Age']

In [10]:
# Checking outliers
Q1 = train['Age'].quantile(0.25)
Q3 = train['Age'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

train = train.drop(train.loc[train['Age'] >= upper].index)

Q1 = train['Fare'].quantile(0.25)
Q3 = train['Fare'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

train = train.drop(train.loc[train['Fare'] >= upper].index)

In [11]:
# sns.boxplot(x=train['Fare'])
# train['Fare'].value_counts().sort_values()

In [12]:
test = test.drop(columns=['Name', 'Surname'])
train = train.drop(columns=['Name', 'Surname'])

In [13]:
train.corr()
# train
# train['Cabin_Number'].value_counts()

# 3 2 1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,size_family,travel_with_family,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,f0,f1,f2,f3
Survived,1.000000,-0.240505,0.500950,-0.174272,-0.003642,0.098420,0.237248,0.085753,-0.037396,0.072365,...,0.124097,0.131946,0.087008,0.024422,-0.025855,-0.207691,0.025416,-0.025276,-0.062979,0.109256
Pclass,-0.240505,1.000000,-0.016664,-0.268402,0.111562,0.081156,-0.585399,0.139745,0.154740,0.087423,...,-0.265017,-0.235577,-0.010532,0.050956,-0.073339,0.603630,0.044150,0.059987,0.063986,-0.088671
Sex,0.500950,-0.016664,1.000000,-0.208441,0.144585,0.288597,0.138300,0.113859,0.148392,0.187453,...,0.046851,0.002688,0.032777,0.106131,-0.024521,-0.026302,-0.100017,0.008940,-0.064000,-0.046572
Age,-0.174272,-0.268402,-0.208441,1.000000,-0.398489,-0.257612,0.002597,-0.008580,-0.303065,-0.326155,...,0.050286,0.100342,-0.085632,-0.079354,0.036709,-0.195855,0.041715,-0.027780,0.020482,0.048074
SibSp,-0.003642,0.111562,0.144585,-0.398489,1.000000,0.409539,0.375000,-0.042389,0.648547,0.493414,...,-0.034355,-0.055912,0.009103,0.004834,-0.017586,0.086622,-0.124002,0.067568,-0.086954,-0.047723
Parch,0.098420,0.081156,0.288597,-0.257612,0.409539,1.000000,0.340954,-0.056970,0.593373,0.456315,...,0.011940,-0.055490,0.051913,0.083275,-0.015678,0.047636,-0.158113,0.029182,0.013558,-0.057652
Fare,0.237248,-0.585399,0.138300,0.002597,0.375000,0.340954,1.000000,-0.085797,0.250083,0.269386,...,0.201053,0.180321,0.012080,-0.022283,0.047257,-0.430327,-0.057415,-0.013402,-0.104626,0.038829
Embarked,0.085753,0.139745,0.113859,-0.008580,-0.042389,-0.056970,-0.085797,1.000000,-0.033754,0.000177,...,-0.048467,0.014117,-0.011902,-0.038168,-0.019047,0.020556,-0.081539,-0.004833,-0.116309,-0.038502
size_family,-0.037396,0.154740,0.148392,-0.303065,0.648547,0.593373,0.250083,-0.033754,1.000000,0.728679,...,-0.046932,-0.033291,0.010604,0.009199,-0.020334,0.021194,-0.125827,0.017142,-0.077877,-0.086507
travel_with_family,0.072365,0.087423,0.187453,-0.326155,0.493414,0.456315,0.269386,0.000177,0.728679,1.000000,...,-0.058940,0.046100,0.055228,0.093262,-0.027905,-0.063469,-0.168180,-0.011812,-0.023740,0.009902


In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE


rfc = RandomForestClassifier(max_depth=500)
abc = AdaBoostClassifier(n_estimators = 5)
gbc = GradientBoostingClassifier(n_estimators = 10)

y = train['Survived']
train = train.drop(columns=['Survived'])

sm = SMOTE(random_state=42)
X, y_res = sm.fit_resample(train, y)
X.shape, y_res.shape

((1018, 28), (1018,))

In [15]:
rfc_cvs = cross_val_score(rfc, X, y_res, cv=5)
print(f"Acurácia média: {rfc_cvs.mean()}")

Acurácia média: 0.8567082005215878


In [16]:
abc_cvs = cross_val_score(abc, X, y_res, cv=5)
print(f"Acurácia média: {abc_cvs.mean()}")

Acurácia média: 0.7987153482082487


In [17]:
gbc_cvs = cross_val_score(gbc, X, y_res, cv=5)
print(f"Acurácia média: {gbc_cvs.mean()}")


Acurácia média: 0.839983579638752


In [18]:
# kf = KFold(n_splits=2, shuffle=True, random_state=0)
# best_score = 0
# save_y_pred = None
# save_X = None
# save_y = None

# for i, (train_idx, test_idx) in enumerate(kf.split(X)):
#     X_train_fold, X_test_fold = X.iloc[train_idx], X.iloc[test_idx]
#     y_train_fold, y_test_fold = y_res.iloc[train_idx], y_res.iloc[test_idx]

#     rfc.fit(X_train_fold, y_train_fold)
#     pred = rfc.predict(X_test_fold)
#     score = rfc.score(X_test_fold, y_test_fold)
    
#     if score > best_score:
#         best_score = score
#         save_y_pred = pred
#         save_X = X_test_fold
#         save_y = y_test_fold
# best_score

In [20]:
gbc.fit(train, y)

,"loss loss: {'log_loss', 'exponential'}, default='log_loss'The loss function to be optimized. 'log_loss' refers to binomial andmultinomial deviance, the same as used in logistic regression.It is a good choice for classification with probabilistic outputs.For loss 'exponential', gradient boosting recovers the AdaBoost algorithm.",'log_loss'
,"learning_rate learning_rate: float, default=0.1Learning rate shrinks the contribution of each tree by `learning_rate`.There is a trade-off between learning_rate and n_estimators.Values must be in the range `[0.0, inf)`.For an example of the effects of this parameter and its interaction with``subsample``, see:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_regularization.py`.",0.1
,"n_estimators n_estimators: int, default=100The number of boosting stages to perform. Gradient boostingis fairly robust to over-fitting so a large number usuallyresults in better performance.Values must be in the range `[1, inf)`.",10
,"subsample subsample: float, default=1.0The fraction of samples to be used for fitting the individual baselearners. If smaller than 1.0 this results in Stochastic GradientBoosting. `subsample` interacts with the parameter `n_estimators`.Choosing `subsample < 1.0` leads to a reduction of varianceand an increase in bias.Values must be in the range `(0.0, 1.0]`.",1.0
,"criterion criterion: {'friedman_mse', 'squared_error'}, default='friedman_mse'The function to measure the quality of a split. Supported criteria are'friedman_mse' for the mean squared error with improvement score byFriedman, 'squared_error' for mean squared error. The default value of'friedman_mse' is generally the best as it can provide a betterapproximation in some cases... versionadded:: 0.18",'friedman_mse'
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, values must be in the range `[2, inf)`.- If float, values must be in the range `(0.0, 1.0]` and `min_samples_split` will be `ceil(min_samples_split * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, values must be in the range `[1, inf)`.- If float, values must be in the range `(0.0, 1.0)` and `min_samples_leaf` will be `ceil(min_samples_leaf * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.Values must be in the range `[0.0, 0.5]`.",0.0
,"max_depth max_depth: int or None, default=3Maximum depth of the individual regression estimators. The maximumdepth limits the number of nodes in the tree. Tune this parameterfor best performance; the best value depends on the interactionof the input variables. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.If int, values must be in the range `[1, inf)`.",3
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.Values must be in the range `[0.0, inf)`.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, `

In [ ]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# import matplotlib.pyplot as plt

# # Gera a matriz usando o melhor fold salvo
# cm = confusion_matrix(save_y, save_y_pred)

# # Plota de forma visual
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Morreu', 'Sobreviveu'])
# disp.plot(cmap='Blues')
# plt.title(f'Matriz de Confusão (Melhor Score: {best_score:.4f})')
# plt.show()

In [28]:
y_pred = gbc.predict(test)


In [29]:
submission = pd.DataFrame({
    "PassengerId": test.index,
    "Survived": y_pred
})

submission.to_csv("submission6.csv", index=False)